In [1]:
import pandas as pd
import sqlite3

data_path = "/Users/eddiewu/Downloads/gu_kelly_xiu/"
covariate_file = "datashare/datashare.csv"
df = pd.read_csv(data_path + covariate_file)


In [2]:
returns_file = "data/tidy_finance_python.sqlite"
tidy_finance = sqlite3.connect(database = data_path + returns_file)


df_returns = pd.read_sql_query(
  sql = "SELECT permno, date, ret_excess FROM crsp_monthly",
  con = tidy_finance,
  parse_dates = {"date"}
)

tidy_finance.close()

In [3]:
df["DATE"] = pd.to_datetime(df["DATE"], format="%Y%m%d")
df["month"] = df["DATE"].dt.to_period("M")
df_returns["month"] = df_returns["date"].dt.to_period("M")


In [4]:
df = df.merge(df_returns, how = 'left', on = ['permno', 'month'])


In [5]:
df.isna().sum()

permno             0
DATE               0
mvel1           3070
beta          400564
betasq        400564
               ...  
zerotrade     309813
sic2          300359
month              0
date          809084
ret_excess    809135
Length: 100, dtype: int64

In [6]:
# Fill NA by cross-sectional median
df = df.groupby("month").apply(
    lambda group: group.fillna(group.median(numeric_only = True))
).reset_index(drop = True)



/var/folders/4n/71vyv5y911sc9sp_7rhzkq1r0000gn/T/ipykernel_31486/3678751893.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("month").apply(


In [8]:
df = df.dropna().reset_index(drop = True)
df['sic2'] = df['sic2'].astype(int)

# Need to standardize data to [-1, 1] within each cross section, for each variable.

In [9]:
# Convert industry code into categorical variable
df = pd.concat([df, pd.get_dummies(df["sic2"], prefix="ind")], axis=1)
df = df.drop(columns=['sic2'])

In [12]:
train_cutoff = '2014-12'
excluded_cols = ['permno', 'DATE', 'month', 'date', 'ret_excess']
target = 'ret_excess'


In [15]:
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.metrics import r2_score


In [16]:
X_train = df[df['month'] <= train_cutoff].drop(columns = excluded_cols)
y_train = df.loc[df['month'] <= train_cutoff, target]

X_test = df[df['month'] > train_cutoff].drop(columns = excluded_cols)
y_test = df.loc[df['month'] > train_cutoff, target]

In [19]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)

r2_linear = r2_score(y_test, y_pred)
print(f"{r2_linear:.4f}")


-0.0053


In [26]:
# lasso = LassoCV(cv = 5, random_state = 0)
lasso = Lasso(alpha = 0.001)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)

r2_lasso = r2_score(y_test, y_pred)
print(f"{r2_linear:.4f}")
print(sum(abs(lasso.coef_) > 1e-8))


-0.0053
27
